In [425]:
import qiskit

In [426]:
qiskit.__qiskit_version__

{'qiskit': '0.46.0', 'qiskit-aer': '0.13.3', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.20.2', 'qiskit-nature': '0.7.1', 'qiskit-finance': '0.4.0', 'qiskit-optimization': '0.6.0', 'qiskit-machine-learning': None}

In [427]:
import random

def generate_unique_random_ints(x, y):
    if x > y + 1:
        raise ValueError("x must be less than or equal to y")
    random_ints = random.sample(range(y), x)
    return random_ints


In [428]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer
from qiskit.circuit import Parameter


def generate_unique_random_pairs(n, x, qulist):
    entanglementpairlist = []
    for i in range(n):
        while True:
            random_control = random.randint(0, x)
            
            while True:
                if random_control in qulist:  #if control qubit is non-zero
                     break
                random_control = random.randint(0, x) #if zero, keep generate new control bit

            random_target = random.randint(0, x) #generate target bit
            qulist.append(random_target)
            while True:
                if random_target != random_control:
                    break
                random_target = random.randint(0, x) #generate a target which is not identical to the control

            entanglement = [random_control, random_target]
            
            pairexist = False
            for sublist in entanglementpairlist: #check whether this pair exists
                if sublist == entanglement:
                    pairexist = True
                    break

            if pairexist == False:
                entanglementpairlist.append(entanglement)
                break;

    return entanglementpairlist



In [429]:
 #quantum circuit with 5 qubits and 4 classical bits
from qiskit.algorithms.optimizers import SPSA, SLSQP, COBYLA
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.quantum_info import Statevector
from qiskit_aer.primitives import Sampler

import random
from math import pi

N = 5
k = N
l = 1
m = 0
t = 10 * 10 ** -3
t_overhead =  1 * 10 ** -3 ##TODO: find toverhead
t_unit = 1000

qulist = []
u3list = []
entanglementlist = []
for j in range(l):
    random_integers = generate_unique_random_ints(k, N) #randomly chose k in 0-3
    qulist += random_integers
    u3list += random_integers

    entanglementpair = generate_unique_random_pairs(m, N-1, qulist)
    entanglementlist += entanglementpair


def buildqc(params):
    pos = 0
    u_pos = 0
    c_pos = 0
    
    qc = QuantumCircuit(N, N) #initialize circuit
    for j in range(l):
        for randomQubit in range(k):
            u1 = params[pos]
            pos += 1
            u2 = params[pos]
            pos += 1
            u3 = params[pos]
            pos += 1
            qc.u(u1, u2, u3, u3list[u_pos]) #add u3 gate
            u_pos += 1
        
        for randomPair in range(m):
            qc.cx(entanglementlist[c_pos][0], entanglementlist[c_pos][1])
            c_pos+=1 #create entanglement
    
        backend = Aer.get_backend('statevector_simulator')
        outputstate = backend.run(qc, shots = 1).result().get_statevector()
        probs = Statevector(outputstate).probabilities()
        return probs


In [430]:
import math
import numpy as np

def CFI(params, t):
    shift = pi/2
    num_runs = N * 2

    shift_list = []
    gradient_list = []

    """create list for shifted params"""
    for i in range(3 * k * l):
        shift_list.append([params[i] - pi/2, params[i] + pi/2])
        params_plus = params.copy()
        params_plus[i] = params[i] + pi/2
        params_minus = params.copy()
        params_minus[i] = params[i] - pi/2
        
        prob_minus = buildqc(params_minus)
        prob_plus = buildqc(params_plus)
    
     
        result = [t ** 2 *(x - y)/2 for x, y in zip(prob_plus, prob_minus)]
        gradient_list.append(result)
        
    CFI = []
    for i in range(3*k*l):
        CFI.append([])
        for j in range(3*k*l):
            CFI[i].append([x * y for x, y in zip(gradient_list[i], gradient_list[j])])
    ##print(CFI)
    return CFI



    ##print(shift_list)
   
##CFI([1,2,3,4])

In [431]:
import numpy as np
import scipy
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.circuit import Parameter
from qiskit.algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler, Estimator

def objective_function(params):
    t_param = params[-1]
    qc_params = params [:-1]
    cfi_matrix = CFI(qc_params, t_param)
    
    result = [[[-element * t_unit * (t_param**2) / (t_overhead + t_param) for element in sublist] for sublist in inner_list] for inner_list in cfi_matrix]
    return result



initial_params = []
temp = []
 
for i in range(3 * k * l):
    initial_params.append(random.uniform(0, 2*np.pi))
    temp.append([0, 2*np.pi])

temp.append([0.0000052, 0.000068])
print(temp)

initial_params.append(t)

# Choose the optimizer
# optimizer = Powell(maxiter=100)
def constraints(params):
    h = []
    for i in range(len(params)-1):
        h.append(2*np.pi- params[i])
    h.append(680 * 10 **-6 -params[-1])
    return h

#construct the bounds in the form of constraints
cons = ({'type': 'ineq', 'fun': constraints})



temp = initial_params.copy()
temp_fun = 0
# Run the optimization with 10 iteration
for i in range(10):
    """trial 0: run with initial"""
    if (i == 0):
        result = scipy.optimize.minimize(objective_function,initial_params,constraints=cons,method='COBYLA')
        print(result.x)
        fun = -result.fun
        if (fun > temp_fun):
            temp_fun = fun
            temp = result.x.copy()
    ##run with previous value
    else:
        result = scipy.optimize.minimize(objective_function,initial_params,constraints=cons,method='COBYLA')
        fun = -result.fun
        if (fun > temp_fun):
            temp_fun = fun
            temp = result.x.copy()

print(temp_fun)
print(temp)

    

[[0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [0, 6.283185307179586], [5.2e-06, 6.8e-05]]
[ 4.72276980e+00  6.27909859e+00  1.37217299e+00  6.26666900e+00
  3.64545456e+00  2.16761663e+00  2.06736847e-02  6.21171724e+00
  3.94126455e+00 -1.46570638e-02  4.03753902e+00  5.75368117e+00
  6.27465173e+00  6.84937072e-01  3.56779459e+00 -9.75203779e-04]
8.666933289717952e-12
[ 4.72276980e+00  6.27909859e+00  1.37217299e+00  6.26666900e+00
  3.64545456e+00  2.16761663e+00  2.06736847e-02  6.21171724e+00
  3.94126455e+00 -1.46570638e-02  4.03753902e+00  5.75368117e+00
  6.27465173e+00  6.84937072e-01  3.56779459e+00 -9.75203779e-04]
